# CASA0007 - Quantitative Methods Group Project
## Group 25

In [5]:
#Reading in libaries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)


## 1. Context of the data

### 1. Cycle to work - Joe
This dataset is from the 2011 Census. Results are based off a Census question that classifies usual residents aged 16 to 74 in England and Wales by their method of travel to work, disaggregated by Census Output Area (OA) and Rural/Urban classification. The estimates are as at census day, 27 March 2011 (single time snapshot).

### 2. Deprivation

### 3. Air Quality - Zach
[Data](https://data.london.gov.uk/dataset/pm2-5-map-and-exposure-data) linked here.


Context about the air quality data

### 4. Cycle Sharing

## 2. Description of data fields/ variables

### 1. Cycle to work - Joe

### 2. Deprivation

### 3. Air Quality - Zach

In [9]:
aq_df = pd.read_csv("data/OA2_PM25_2013.csv")

### 4. Cycle Sharing

## 3. Summary statistic fields

## 4. Appropriate plots to communicate distribution

## 5. Appropriate plots to illustrate relationship between key fields